In [1]:
# main.py

from config import *
from utils.seed import set_seed
from data.splits import create_splits
from data.transforms import get_transforms
from data.datamodule import ImageDataModule
from models.efficientnet import EfficientNetLit

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from torch.utils.data import WeightedRandomSampler

import torch
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

from utils.make_sampler import make_sampler



def main():
    set_seed(SEED)

    train_df, val_df, test_df, classes = create_splits(
        split_csv=PROJECT_DIR / "experiment/BCN20000/master_split_file_3.csv",
        images_dir=METADATA_DIR,
    )


    #train_df, val_df, test_df, classes = create_splits(
    #    metadata_csv= METADATA_DIR,
    #    images_dir= METADATA_DIR,
    #    seed=SEED,
    #    test_size=TEST_SIZE,
    #    val_size=VAL_SIZE,
    #)

    print(
        len(train_df),
        len(val_df),
        len(test_df),
        train_df.lesion_id.nunique(),
        val_df.lesion_id.nunique(),
        test_df.lesion_id.nunique(),
    )

    print("TRAIN class distribution:")
    print(train_df["label"].value_counts().sort_index())
    
    train_sampler = make_sampler(train_df["label"].values)

    train_tfms, val_tfms = get_transforms()

    datamodule = ImageDataModule(
        train_df=train_df,
        val_df=val_df,
        test_df=test_df,
        train_tfms=train_tfms,
        val_tfms=val_tfms,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        train_sampler=train_sampler,
     )

    model = EfficientNetLit(
        num_classes=len(classes),
        lr=LR,
        weight_decay=WEIGHT_DECAY,
    )


    logger = CSVLogger("logs", name="efficientnet")

    early_stop = pl.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=20,
        mode="min",
        verbose=True,
    )


    trainer = pl.Trainer(
        max_epochs=EPOCHS,
        accelerator="gpu",
        devices=1,
        logger=logger,
        callbacks=[
            ModelCheckpoint(
                monitor="val_loss",
                mode="min",
            ),
            early_stop,
        ],
    )

    trainer.fit(model, datamodule=datamodule)


if __name__ == "__main__":
    main()


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


13504 660 3475 3821 202 1006
TRAIN class distribution:
label
0    2838
1     126
2     480
3    3049
4    4321
5     244
6     988
7     211
8     816
9     431
Name: count, dtype: int64


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name       ┃ Type             ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ encoder    │ EfficientNet     │  4.0 M │ train │     0 │
│ 1 │ classifier │ Sequential       │  826 K │ train │     0 │
│ 2 │ criterion  │ CrossEntropyLoss │      0 │ train │     0 │
└───┴────────────┴──────────────────┴────────┴───────┴───────┘

Trainable params: 4.8 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.8 M                                                                                                
Total estimated model params size (MB): 19                                                                         
Modules in train mode: 344                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

Metric val_loss improved. New best score: 1.440
Metric val_loss improved by 0.027 >= min_delta = 0.0. New best score: 1.413
Metric val_loss improved by 0.008 >= min_delta = 0.0. New best score: 1.405
Metric val_loss improved by 0.081 >= min_delta = 0.0. New best score: 1.324

Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

/home/.conda/envs/env/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
